## 1 导入model

In [20]:
# 导入LLM
# 初始化模型
from langchain_openai import ChatOpenAI
import os
# 1. 导入LLM
# 初始化模型
model = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称
)



## 2 导入Embedding模型

In [4]:
# 2. 导入Embedding模型
from langchain_community.embeddings import DashScopeEmbeddings

embeddings = DashScopeEmbeddings(model = "text-embedding-v3")

## 3 构建向量库

In [6]:
# 3. 构建向量库
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# 4 RAG链

In [7]:
# 4. RAG链
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict


USER_AGENT environment variable not set, consider setting it to identify your requests.


## 4.1 文档加载器和导入文档

In [24]:
# # 4.1 文档加载器和导入文档
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# 4.1.2 PDF
from langchain_community.document_loaders import PyPDFLoader

# 加载PDF文档
loader = PyPDFLoader("ntk.pdf")
docs = loader.load()


## 4.2 文本分割

In [25]:
# 4.2 文本切割器和分割文档
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_spliter.split_documents(docs)


In [26]:
print(all_splits[0])

page_content='Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE FISCAL YEAR ENDED MAY 31, 2023
OR
☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE TRANSITION PERIOD FROM                         TO                         .
Commission File No. 1-10635
NIKE, Inc.
(Exact name of Registrant as specified in its charter)
Oregon 93-0584541
(State or other jurisdiction of incorporation) (IRS Employer Identification No.)
One Bowerman Drive, Beaverton, Oregon 97005-6453
(Address of principal executive offices and zip code)
(503) 671-6453
(Registrant's telephone number, including area code)
SECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:
Class B Common Stock NKE New York Stock Exchange
(Title of each class) (Trading symbol) (Name of each exchange on which registered)' metadata={'p

## 4.3 构建索引
在RAG（检索增强生成）系统中，这是一个关键步骤，它会：

1. 对每个文档片段使用嵌入模型生成向量表示
2. 将这些向量和对应的文档内容存储在向量数据库中
3. 建立索引以便后续快速检索

这样，当用户提出问题时，系统就可以通过向量相似度搜索，快速找到与问题最相关的文档片段，用于增强生成回答。

In [27]:
# 4.3 构建索引
_ = vector_store.add_documents(documents=all_splits)

## 4.4 构建RAG的Prompt模版
1. 从LangChain Hub拉取预定义的RAG提示模板

In [29]:
# 4.4 构建RAG的Prompt模版
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

## 4.5 基于知识库的问答系统
1. State 用于存储应用当前的状态，包括用户问题、检索到的上下文文档和最终答案。
2. 这个函数接收当前状态（含用户问题），调用 vector_store.similarity_search 检索相关文档，返回新的 context。vector_store 是你的向量数据库对象，比如 FAISS、Milvus、Chroma 等。
3. 将检索到的文档内容拼接，和问题一起传递给 prompt，生成用于大模型的输入。
model.invoke(messages) 调用大模型（如 OpenAI GPT、Llama 等）生成答案。
返回生成的答案。

In [30]:
# 4.5 
# Define state for application
# 4.5.1 State 用于存储应用当前的状态，包括用户问题、检索到的上下文文档和最终答案。
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# 4.5.2 
# 这个函数接收当前状态（含用户问题），调用 vector_store.similarity_search 检索相关文档，返回新的 context。
# vector_store 是你的向量数据库对象，比如 FAISS、Milvus、Chroma 等。
# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


# 4.5.3
# 将检索到的文档内容拼接，和问题一起传递给 prompt，生成用于大模型的输入。
# model.invoke(messages) 调用大模型（如 OpenAI GPT、Llama 等）生成答案。
# 返回生成的答案。
def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model.invoke(messages)
    return {"answer": response.content}


## 4.6 如何用StateGraph来编排和编译应用的流程，并为测试做准备
1. StateGraph(State)

    创建一个以State为状态类型的有向图对象，用于管理应用的执行流程。
    用来构建你的RAG流程（可以很灵活地加节点、分支等）。

2. add_sequence([retrieve, generate])

    按顺序添加两个步骤：retrieve（检索）和generate（生成）。
    按顺序安排每个处理步骤（如先检索后生成）。

    这意味着执行流程会自动从retrieve流向generate。

3. add_edge(START, "retrieve")

    显式指定流程从START节点进入retrieve步骤。
    指定流程的起点或节点之间的连接关系。

4. compile()

    编译成可执行的“问答引擎”对象。
    编译流程图，得到可以直接运行的graph对象。

5. graph.invoke()

    直接用来问问题，得到答案。

In [31]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

# 5 RAG调用


In [32]:
response = graph.invoke({"question": "这是关于什么的PDF？"})
print(response["answer"])

这份PDF是关于耐克公司（NIKE, INC.）的年度报告，具体包含了截至2023年5月31日的财务报表、综合收益表、现金流量表以及股东权益表等信息。此外，还提供了审计师独立意见和财务注释。这是向美国证券交易委员会提交的10-K表格文件。


In [ ]:
import streamlit as st

st.title("PDF/网页文档智能问答")

user_input = st.text_input("请输入你的问题：")

if st.button("提交") and user_input:
    with st.spinner("正在生成答案..."):
        response = graph.invoke({"question": user_input})
        st.write("答案：")
        st.write(response["answer"])
